1. Imports + Set Up

In [17]:
import yfinance as yf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


2. Stock + News

In [18]:
def get_stock_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    df['daily_return'] = df['Close'].pct_change()
    df['volume_change'] = df['Volume'].pct_change()
    return df

def get_news_articles(query, from_date, to_date, api_key):
    url = (
        f'https://newsapi.org/v2/everything?q={query}&from={from_date}&to={to_date}'
        f'&language=en&sortBy=publishedAt&pageSize=100&apiKey={api_key}'
    )
    response = requests.get(url)
    data = response.json()
    return data.get('articles', [])

3. Sentiment

In [19]:
def analyze_sentiment(articles):
    records = []

    for article in articles:
        text = (article.get("title", "") or "") + " " + (article.get("description", "") or "")
        if not text.strip():
            continue

        try:
            result = sentiment_pipeline(text[:512])[0]  # Truncate to 512 tokens max
            label = result["label"]
            score = {"Positive": 1, "Neutral": 0, "Negative": -1}.get(label, 0)
            date = article['publishedAt'][:10]
            records.append((date, score))
        except Exception as e:
            print(f"Skipping article due to error: {e}")
            continue

    df = pd.DataFrame(records, columns=["date", "sentiment"])
    df["date"] = pd.to_datetime(df["date"])
    return df.groupby("date").mean()

4. Model

In [20]:
def merge_and_label(stock_df, sentiment_df):
    df = stock_df.merge(sentiment_df, how='left', left_index=True, right_index=True)
    df['sentiment'] = df['sentiment'].fillna(0)
    df['target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    return df.dropna()

def train_and_evaluate(df):
    features = ['sentiment', 'daily_return', 'volume_change']
    X = df[features]
    y = df['target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))
    return model

5. Pipeline

In [21]:
# Params
ticker = "MSFT"
start_date = "2023-01-01"
end_date = "2024-01-01"
api_key = "90d80862968d439ab1f7cb56d1c054a3"

# Run
stock_df = get_stock_data(ticker, start_date, end_date)
news_articles = get_news_articles(ticker, start_date, end_date, api_key)
sentiment_df = analyze_sentiment(news_articles)
merged_df = merge_and_label(stock_df, sentiment_df)
model = train_and_evaluate(merged_df)

Failed to get ticker 'MSFT' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MSFT']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.